In [ ]:
import json
import dateutil.parser as parser
from statistics import mean, median
import copy

path_to_file = r'data\revall_analysis.json'
path_to_repository_info = r'data\repository_info.json'

seconds_in_a_day = 24*3600
with open(path_to_file, 'r') as f:
    recall_data = json.load(f)


In [ ]:
all_subset = recall_data
npm_subset = {k:v for k,v in recall_data.items() if 'ecosystem' in v and 'npm' in v['ecosystem']}
pypi_subset = {k:v for k,v in recall_data.items() if 'ecosystem' in v and 'pypi' in v['ecosystem']}
mvn_subset = {k:v for k,v in recall_data.items() if 'ecosystem' in v and 'maven' in v['ecosystem']}

In [ ]:
def calculate_delay_no_separation(delays_data_subset, date_key, label):
    for x in delays_data_subset:
        if 'creation_date' in delays_data_subset[x] and delays_data_subset[x]['creation_date'] and date_key in delays_data_subset[x] and delays_data_subset[x][date_key]:
            creation_date = parser.parse(delays_data_subset[x]['creation_date'])
            ref_date = min([parser.parse(y[1])for y in delays_data_subset[x][date_key]])
            diff = creation_date-ref_date
            delays = diff.total_seconds()/seconds_in_a_day
            delays_data_subset[x][f'{date_key}_diff'] = delays
        
    ids_all = len([1 for x in delays_data_subset if 'creation_date' in delays_data_subset[x] and delays_data_subset[x]['creation_date']])
    ids_with_diffs = len([1 for x in delays_data_subset if f'{date_key}_diff' in delays_data_subset[x]])
    ids_with_positive_diffs = len([1 for x in delays_data_subset if f'{date_key}_diff' in delays_data_subset[x] and delays_data_subset[x][f'{date_key}_diff']>0])
    ids_with_week_old_diffs = len([1 for x in delays_data_subset if f'{date_key}_diff' in delays_data_subset[x] and delays_data_subset[x][f'{date_key}_diff']>7])

    diffs = [delays_data_subset[x][f'{date_key}_diff'] for x in delays_data_subset if f'{date_key}_diff' in delays_data_subset[x]]

    print(label)
    print(date_key)
    print('count', ids_with_diffs)
    print('median', median(diffs))
    print('recall at 0', ids_with_positive_diffs/ids_all)
    print('recall at 7', ids_with_week_old_diffs/ids_all)

In [ ]:
def calculate_delays_vs_with_issues(delays_data_subset, date_key, label):
    for x in delays_data_subset:
        if 'creation_date' in delays_data_subset[x] and delays_data_subset[x]['creation_date'] and date_key in delays_data_subset[x] and delays_data_subset[x][date_key]:
            creation_date = parser.parse(delays_data_subset[x]['creation_date'])
            ref_date = min([parser.parse(y['ref_date'])for y in delays_data_subset[x][date_key]])
            # ref_date = min([parser.parse(y[1])for y in delays_data_subset[x][date_key]])
            diff = creation_date-ref_date
            delays = diff.total_seconds()/seconds_in_a_day
            delays_data_subset[x][f'{date_key}_diff'] = delays
        
    ids_with_refs = len([1 for x in delays_data_subset if ('first_issue' in delays_data_subset[x] and delays_data_subset[x]['first_issue'])])
    ids_with_diffs = len([1 for x in delays_data_subset if f'{date_key}_diff' in delays_data_subset[x]])
    ids_with_positive_diffs = len([1 for x in delays_data_subset if f'{date_key}_diff' in delays_data_subset[x] and delays_data_subset[x][f'{date_key}_diff']>0])
    ids_with_week_old_diffs = len([1 for x in delays_data_subset if f'{date_key}_diff' in delays_data_subset[x] and delays_data_subset[x][f'{date_key}_diff']>7])

    diffs = [delays_data_subset[x][f'{date_key}_diff'] for x in delays_data_subset if f'{date_key}_diff' in delays_data_subset[x]]

    print(label)
    print(date_key)
    print('count', ids_with_diffs)
    print('median', median(diffs))
    print('recall at 0', ids_with_positive_diffs/ids_with_refs)
    print('recall at 7', ids_with_week_old_diffs/ids_with_refs)
    print('recall at 30', ids_with_week_old_diffs/ids_with_refs)

In [ ]:
def calculate_delays_vs_with_refs(delays_data_subset, date_key, label):
    for x in delays_data_subset:
        if 'creation_date' in delays_data_subset[x] and delays_data_subset[x]['creation_date'] and date_key in delays_data_subset[x] and delays_data_subset[x][date_key]:
            creation_date = parser.parse(delays_data_subset[x]['creation_date'])
            ref_date = min([parser.parse(y['ref_date'])for y in delays_data_subset[x][date_key]])
            # ref_date = min([parser.parse(y[1])for y in delays_data_subset[x][date_key]])
            diff = creation_date-ref_date
            delays = diff.total_seconds()/seconds_in_a_day
            delays_data_subset[x][f'{date_key}_diff'] = delays
        
    ids_with_refs = len([1 for x in delays_data_subset if ('first_ref' in delays_data_subset[x] and delays_data_subset[x]['first_ref']) or ('first_commit' in delays_data_subset[x] and delays_data_subset[x]['first_commit']) or ('first_ref' in delays_data_subset[x] and delays_data_subset[x]['first_ref'])])
    ids_with_diffs = len([1 for x in delays_data_subset if f'{date_key}_diff' in delays_data_subset[x]])
    ids_with_positive_diffs = len([1 for x in delays_data_subset if f'{date_key}_diff' in delays_data_subset[x] and delays_data_subset[x][f'{date_key}_diff']>0])
    ids_with_week_old_diffs = len([1 for x in delays_data_subset if f'{date_key}_diff' in delays_data_subset[x] and delays_data_subset[x][f'{date_key}_diff']>7])

    diffs = [delays_data_subset[x][f'{date_key}_diff'] for x in delays_data_subset if f'{date_key}_diff' in delays_data_subset[x]]

    print(label)
    print(date_key)
    print('count', ids_with_diffs)
    print('median', median(diffs))
    print('recall at 0', ids_with_positive_diffs/ids_with_refs)
    print('recall at 7', ids_with_week_old_diffs/ids_with_refs)
    print('recall at 30', ids_with_week_old_diffs/ids_with_refs)

In [ ]:
def calculate_delays_vs_with_commits(delays_data_subset, date_key, label):
    for x in delays_data_subset:
        if 'creation_date' in delays_data_subset[x] and delays_data_subset[x]['creation_date'] and date_key in delays_data_subset[x] and delays_data_subset[x][date_key]:
            creation_date = parser.parse(delays_data_subset[x]['creation_date'])
            ref_date = min([parser.parse(y['ref_date'])for y in delays_data_subset[x][date_key]])
            # ref_date = min([parser.parse(y[1])for y in delays_data_subset[x][date_key]])
            diff = creation_date-ref_date
            delays = diff.total_seconds()/seconds_in_a_day
            delays_data_subset[x][f'{date_key}_diff'] = delays
        
    ids_with_refs = len([1 for x in delays_data_subset if ('first_commit' in delays_data_subset[x] and delays_data_subset[x]['first_commit'])])
    ids_with_diffs = len([1 for x in delays_data_subset if f'{date_key}_diff' in delays_data_subset[x]])
    ids_with_positive_diffs = len([1 for x in delays_data_subset if f'{date_key}_diff' in delays_data_subset[x] and delays_data_subset[x][f'{date_key}_diff']>0])
    ids_with_week_old_diffs = len([1 for x in delays_data_subset if f'{date_key}_diff' in delays_data_subset[x] and delays_data_subset[x][f'{date_key}_diff']>7])

    diffs = [delays_data_subset[x][f'{date_key}_diff'] for x in delays_data_subset if f'{date_key}_diff' in delays_data_subset[x]]

    print(label)
    print(date_key)
    print('count', ids_with_diffs)
    print('median', median(diffs))
    print('recall at 0', ids_with_positive_diffs/ids_with_refs)
    print('recall at 7', ids_with_week_old_diffs/ids_with_refs)
    print('recall at 30', ids_with_week_old_diffs/ids_with_refs)

In [ ]:
def calculate_delays_vs_with_commits_and_all_refs(delays_data_subset, date_key, label):
    for x in delays_data_subset:
        if 'creation_date' in delays_data_subset[x] and delays_data_subset[x]['creation_date'] and date_key in delays_data_subset[x] and delays_data_subset[x][date_key]:
            creation_date = parser.parse(delays_data_subset[x]['creation_date'])
            ref_date = min([parser.parse(y['ref_date'])for y in delays_data_subset[x][date_key]])
            # ref_date = min([parser.parse(y[1])for y in delays_data_subset[x][date_key]])
            diff = creation_date-ref_date
            delays = diff.total_seconds()/seconds_in_a_day
            delays_data_subset[x][f'{date_key}_diff'] = delays
        
    ids_with_refs = len([1 for x in delays_data_subset if ('first_commit' in delays_data_subset[x] and delays_data_subset[x]['first_commit']) and ('all_refs' in delays_data_subset[x] and delays_data_subset[x]['all_refs'])])
    ids_with_diffs = len([1 for x in delays_data_subset if f'{date_key}_diff' in delays_data_subset[x]])
    ids_with_positive_diffs = len([1 for x in delays_data_subset if f'{date_key}_diff' in delays_data_subset[x] and delays_data_subset[x][f'{date_key}_diff']>0])
    ids_with_week_old_diffs = len([1 for x in delays_data_subset if f'{date_key}_diff' in delays_data_subset[x] and delays_data_subset[x][f'{date_key}_diff']>7])

    diffs = [delays_data_subset[x][f'{date_key}_diff'] for x in delays_data_subset if f'{date_key}_diff' in delays_data_subset[x]]

    print(label)
    print(date_key)
    print('count', ids_with_diffs)
    print('median', median(diffs))
    print('recall at 0', ids_with_positive_diffs/ids_with_refs)
    print('recall at 7', ids_with_week_old_diffs/ids_with_refs)
    print('recall at 30', ids_with_week_old_diffs/ids_with_refs)

def calculate_delays_vs_all_refs_field(delays_data_subset, date_key, label):
    for x in delays_data_subset:
        if 'creation_date' in delays_data_subset[x] and delays_data_subset[x]['creation_date'] and date_key in delays_data_subset[x] and delays_data_subset[x][date_key]:
            creation_date = parser.parse(delays_data_subset[x]['creation_date'])
            ref_date = min([parser.parse(y['ref_date'])for y in delays_data_subset[x][date_key]])
            # ref_date = min([parser.parse(y[1])for y in delays_data_subset[x][date_key]])
            diff = creation_date-ref_date
            delays = diff.total_seconds()/seconds_in_a_day
            delays_data_subset[x][f'{date_key}_diff'] = delays
        
    ids_with_refs = len([1 for x in delays_data_subset if ('all_refs' in delays_data_subset[x] and delays_data_subset[x]['all_refs'])])
    ids_with_diffs = len([1 for x in delays_data_subset if f'{date_key}_diff' in delays_data_subset[x]])
    ids_with_positive_diffs = len([1 for x in delays_data_subset if f'{date_key}_diff' in delays_data_subset[x] and delays_data_subset[x][f'{date_key}_diff']>0])
    ids_with_week_old_diffs = len([1 for x in delays_data_subset if f'{date_key}_diff' in delays_data_subset[x] and delays_data_subset[x][f'{date_key}_diff']>7])

    diffs = [delays_data_subset[x][f'{date_key}_diff'] for x in delays_data_subset if f'{date_key}_diff' in delays_data_subset[x]]

    print(label)
    print(date_key)
    print('count', ids_with_diffs)
    print('median', median(diffs))
    print('recall at 0', ids_with_positive_diffs/ids_with_refs)
    print('recall at 7', ids_with_week_old_diffs/ids_with_refs)
    print('recall at 30', ids_with_week_old_diffs/ids_with_refs)

def calculate_delays_vs_with_issues_and_all_refs(delays_data_subset, date_key, label):
    for x in delays_data_subset:
        if 'creation_date' in delays_data_subset[x] and delays_data_subset[x]['creation_date'] and date_key in delays_data_subset[x] and delays_data_subset[x][date_key]:
            creation_date = parser.parse(delays_data_subset[x]['creation_date'])
            ref_date = min([parser.parse(y['ref_date'])for y in delays_data_subset[x][date_key]])
            # ref_date = min([parser.parse(y[1])for y in delays_data_subset[x][date_key]])
            diff = creation_date-ref_date
            delays = diff.total_seconds()/seconds_in_a_day
            delays_data_subset[x][f'{date_key}_diff'] = delays
        
    ids_with_refs = len([1 for x in delays_data_subset if ('first_issue' in delays_data_subset[x] and delays_data_subset[x]['first_issue']) and ('all_refs' in delays_data_subset[x] and delays_data_subset[x]['all_refs'])])
    ids_with_diffs = len([1 for x in delays_data_subset if f'{date_key}_diff' in delays_data_subset[x]])
    ids_with_positive_diffs = len([1 for x in delays_data_subset if f'{date_key}_diff' in delays_data_subset[x] and delays_data_subset[x][f'{date_key}_diff']>0])
    ids_with_week_old_diffs = len([1 for x in delays_data_subset if f'{date_key}_diff' in delays_data_subset[x] and delays_data_subset[x][f'{date_key}_diff']>7])

    diffs = [delays_data_subset[x][f'{date_key}_diff'] for x in delays_data_subset if f'{date_key}_diff' in delays_data_subset[x]]

    print(label)
    print(date_key)
    print('count', ids_with_diffs)
    print('median', median(diffs))
    print('recall at 0', ids_with_positive_diffs/ids_with_refs)
    print('recall at 7', ids_with_week_old_diffs/ids_with_refs)
    print('recall at 30', ids_with_week_old_diffs/ids_with_refs)
    

In [ ]:
def calculate_delays_vs_classified_commits(delays_data_subset, date_key, label):
    for x in delays_data_subset:
        if date_key not in delays_data_subset[x] or delays_data_subset[x][date_key] == None or not delays_data_subset[x][date_key]:
            continue
            
        if 'creation_date' not in delays_data_subset[x] or delays_data_subset[x]['creation_date'] == None:
            continue


        creation_date = parser.parse(delays_data_subset[x]['creation_date'])
        dates = [matched_ref['ref_date'] for matched_ref in delays_data_subset[x][date_key] if matched_ref['ref_date'] != 'not_spotted']

        if len(dates) > 0:
            diff = creation_date-min([parser.parse(d) for d in dates])
            delays = diff.total_seconds()/seconds_in_a_day
            delays_data_subset[x][f'{date_key}_diff'] = delays
        
    ids_all = len([1 for x in delays_data_subset if f'{date_key}_diff' in delays_data_subset[x] or \
        (date_key in delays_data_subset[x] and delays_data_subset[x][date_key] != None and delays_data_subset[x][date_key])])
    ids_with_diffs = len([1 for x in delays_data_subset if f'{date_key}_diff' in delays_data_subset[x]])
    ids_with_positive_diffs = len([1 for x in delays_data_subset if f'{date_key}_diff' in delays_data_subset[x] and delays_data_subset[x][f'{date_key}_diff']>0])
    ids_with_week_old_diffs = len([1 for x in delays_data_subset if f'{date_key}_diff' in delays_data_subset[x] and delays_data_subset[x][f'{date_key}_diff']>7])

    diffs = [delays_data_subset[x][f'{date_key}_diff'] for x in delays_data_subset if f'{date_key}_diff' in delays_data_subset[x]]

    print()
    print(date_key)
    print(label)
    print('count', ids_with_diffs)
    print('median', median(diffs))
    print('recall at 0',ids_with_positive_diffs,  ids_with_positive_diffs/ids_all)
    print('recall at 7',ids_with_week_old_diffs, ids_with_week_old_diffs/ids_all)

Recall for 'Data' chapter

In [ ]:
def first_ref(delays_data_subset, label):
    calculate_delay_no_separation(delays_data_subset, 'first_ref', label)

first_ref(recall_data, 'All')
first_ref(npm_subset, 'npm')
first_ref(pypi_subset, 'pypi')
first_ref(mvn_subset, 'mvn')

In [ ]:
def first_issue(delays_data_subset, label):
    calculate_delay_no_separation(delays_data_subset, 'first_issue', label)

first_issue(recall_data, 'All')
first_issue(npm_subset, 'npm')
first_issue(pypi_subset, 'pypi')
first_issue(mvn_subset, 'mvn')

In [ ]:
def first_commit(delays_data_subset, label):
    calculate_delay_no_separation(delays_data_subset, 'first_commit', label)

first_commit(recall_data, 'All')
first_commit(npm_subset, 'npm')
first_commit(pypi_subset, 'pypi')
first_commit(mvn_subset, 'mvn')

Recall for Experiments 1 & 2

In [ ]:
def first_sec_phrase(delays_data_subset, label):
    calculate_delays_vs_with_refs(delays_data_subset, 'first_sec_phrase', label)

first_sec_phrase(recall_data, 'All')
first_sec_phrase(npm_subset, 'npm')
first_sec_phrase(pypi_subset, 'pypi')
first_sec_phrase(mvn_subset, 'mvn')

In [ ]:
def first_sec_label(delays_data_subset, label):
    calculate_delays_vs_with_issues(delays_data_subset, 'first_sec_label', label)

first_sec_label(recall_data, 'All')
first_sec_label(npm_subset, 'npm')
first_sec_label(pypi_subset, 'pypi')
first_sec_label(mvn_subset, 'mvn')

In [ ]:
def fb_commit_classification(delays_data_subsetx, label):
    calculate_delays_vs_classified_commits(delays_data_subsetx, 'fb_commit_classification', label)

fb_commit_classification(all_subset, 'All')
fb_commit_classification(npm_subset, 'npm')
fb_commit_classification(pypi_subset, 'pypi')
fb_commit_classification(mvn_subset,  'mvn')

In [ ]:
def vfm_combined_classification(delays_data_subset, label):
    calculate_delays_vs_classified_commits(delays_data_subset, 'vfm_combined_classification', label)

vfm_combined_classification(all_subset, 'All')
vfm_combined_classification(npm_subset, 'npm')
vfm_combined_classification(pypi_subset, 'pypi')
vfm_combined_classification(mvn_subset,  'mvn')

Evaluation: Disclosure Year

In [ ]:
with open(path_to_file, 'r') as f:
    recall_data = json.load(f)

with_creation_dates = {k:v for k,v in recall_data.items() if 'creation_date' in v and v['creation_date']}
for x in with_creation_dates:
    with_creation_dates[x]['year'] = parser.parse(with_creation_dates[x]['creation_date']).year

all_2017 = {k:v for k,v in recall_data.items() if 'year' in v and v['year']==2017}
all_2018 = {k:v for k,v in recall_data.items() if 'year' in v and v['year']==2018}
all_2019 = {k:v for k,v in recall_data.items() if 'year' in v and v['year']==2019}
all_2020 = {k:v for k,v in recall_data.items() if 'year' in v and v['year']==2020}
all_2021 = {k:v for k,v in recall_data.items() if 'year' in v and v['year']==2021}
all_2022 = {k:v for k,v in recall_data.items() if 'year' in v and v['year']==2022}

In [ ]:
selected_year = all_2021
selected_year_s = '2021'

first_sec_phrase(selected_year, selected_year_s)
first_sec_label(selected_year, selected_year_s)
fb_commit_classification(selected_year, selected_year_s)
vfm_combined_classification(selected_year, selected_year_s)

Evaluation: Severity

In [ ]:
with open(path_to_file, 'r') as f:
    recall_data = json.load(f)

all_low =      {k:v for k,v in recall_data.items() if 'severity' in v and v['severity']=='LOW'}
all_medium =   {k:v for k,v in recall_data.items() if 'severity' in v and v['severity']=='MEDIUM'}
all_high =     {k:v for k,v in recall_data.items() if 'severity' in v and v['severity']=='HIGH'}
all_critical = {k:v for k,v in recall_data.items() if 'severity' in v and v['severity']=='CRITICAL'}

In [ ]:

selected_crticality = all_high
selected_crticality_s = 'all_high'
first_sec_phrase(selected_crticality, selected_crticality_s)
first_sec_label(selected_crticality, selected_crticality_s)
fb_commit_classification(selected_crticality, selected_crticality_s)
vfm_combined_classification(selected_crticality, selected_crticality_s)

Evaluation: Commit size

In [ ]:
bottom_threshold = 10
top_threshold = 40

with open(path_to_file, 'r') as f:
    recall_data = json.load(f)

def split_by_commit_size(data_subset, data_keys):
    result_bot = copy.deepcopy(data_subset)
    result_mid = copy.deepcopy(data_subset)
    result_top = copy.deepcopy(data_subset)
    for k,v in data_subset.items():
        for data_key in data_keys:
            if data_key not in v or v[data_key] == None:
                continue

            commit_size_bot = [matched_ref for matched_ref in v[data_key] if 'ref_commit_size' in matched_ref and matched_ref['ref_commit_size'] != None and matched_ref['ref_commit_size'] <= bottom_threshold]
            commit_size_mid = [matched_ref for matched_ref in v[data_key] if 'ref_commit_size' in matched_ref and matched_ref['ref_commit_size'] != None and matched_ref['ref_commit_size'] > bottom_threshold and matched_ref['ref_commit_size'] <= top_threshold]
            commit_size_top = [matched_ref for matched_ref in v[data_key] if 'ref_commit_size' in matched_ref and matched_ref['ref_commit_size'] != None and matched_ref['ref_commit_size'] > top_threshold]

            result_bot[k][data_key] = list(commit_size_bot)
            result_mid[k][data_key] = list(commit_size_mid)
            result_top[k][data_key] = list(commit_size_top)
        
    return result_bot,result_mid,result_top

data_keys=[
    'all_refs',
    'first_sec_phrase',
    'fb_commit_classification',
    'vfm_combined_classification'
]

all_bot_commit_size, all_mid_commit_size, all_top_commit_size = split_by_commit_size(recall_data, data_keys)

In [ ]:
selected_commit_size = all_top_commit_size
selected_commit_size_s = 'all_top_commit_size'
calculate_delays_vs_with_commits_and_all_refs(selected_commit_size, 'first_sec_phrase', selected_commit_size_s)
# first_sec_label(selected_commit_size, selected_commit_size_s)
fb_commit_classification(selected_commit_size, selected_commit_size_s)
vfm_combined_classification(selected_commit_size, selected_commit_size_s)


Evaluation: Repository stars

In [ ]:
bottom_threshold = 400
top_threshold = 3200

with open(path_to_file, 'r') as f:
    recall_data = json.load(f)

with open(path_to_repository_info, 'r') as f:
    repository_info = json.load(f)

def split_by_repository_stars(data_subset, data_keys):
    result_bot = copy.deepcopy(data_subset)
    result_mid = copy.deepcopy(data_subset)
    result_top = copy.deepcopy(data_subset)
    for k,v in data_subset.items():
        for data_key in data_keys:
            if data_key not in v or v[data_key] == None:
                continue

            commit_size_bot = []
            commit_size_mid = []
            commit_size_top = []
            for matched_ref in v[data_key]:
                if 'ref_repo_full_name' not in matched_ref:
                    continue
                if matched_ref['ref_repo_full_name'] not in repository_info:
                    continue

                stars_count = repository_info[matched_ref['ref_repo_full_name']]['repository_info']['stargazers_count']
                if stars_count <=bottom_threshold:
                    commit_size_bot.append(matched_ref)
                elif stars_count > bottom_threshold and stars_count <= top_threshold:
                    commit_size_mid.append(matched_ref)
                elif stars_count > top_threshold:
                    commit_size_top.append(matched_ref)

            result_bot[k][data_key] = commit_size_bot
            result_mid[k][data_key] = commit_size_mid
            result_top[k][data_key] = commit_size_top
        
    return result_bot,result_mid,result_top

data_keys=[
    'all_refs',
    'first_sec_phrase',
    'first_sec_label',
    'fb_commit_classification',
    'vfm_combined_classification'
]

all_bot_repo_stars, all_mid_repo_stars, all_top_repo_stars = split_by_repository_stars(recall_data, data_keys)

In [ ]:
selected_repo_class = all_bot_repo_stars
selected_repo_class_s = 'all_bot_repo_stars'
calculate_delays_vs_all_refs_field(selected_repo_class, 'first_sec_phrase', selected_repo_class_s)
calculate_delays_vs_with_issues_and_all_refs(selected_repo_class, 'first_sec_label', selected_repo_class_s)
fb_commit_classification(selected_repo_class, selected_repo_class_s)
vfm_combined_classification(selected_repo_class, selected_repo_class_s)


In [ ]:
print('ok')